# Model

In [1]:
import pandas as pd
from database import engine

from IPython.display import HTML

## Loading data

In [2]:
user_data = pd.read_sql(
    "SELECT * FROM public.user_data LIMIT 10000;", 
    con = engine,
    index_col = "user_id"
)
post_data = pd.read_sql(
    "SELECT * FROM public.post_text_df LIMIT 10000;",
    con = engine,
    index_col = "post_id"
)
feed_data = pd.read_sql(
    "SELECT * FROM public.feed_data limit 10000;",
    con = engine
)

# joined data is where each user 
# is matched with all his actions
query = """
SELECT 
    public.feed_data.timestamp,
    public.feed_data.user_id,
    public.feed_data.post_id,
    public.feed_data.action,
    public.feed_data.target,
    public.user_data.gender,
    public.user_data.age,
    public.user_data.country,
    public.user_data.city,
    public.user_data.exp_group,
    public.user_data.os,
    public.user_data.source,
    public.post_text_df.text,
    public.post_text_df.topic
FROM public.feed_data
LEFT JOIN public.user_data
    ON public.feed_data.user_id = public.user_data.user_id
LEFT JOIN public.post_text_df
    ON public.feed_data.post_id = public.post_text_df.post_id
LIMIT 10000;
"""
joined_data = pd.read_sql(query, con = engine)

In [3]:
df_show = {
    "Users data" : user_data,
    "Post data" : post_data,
    "Feed data" : feed_data,
    "Joined data" : joined_data
}

for title, df in df_show.items():
    display(HTML(f"<h3>{title}</h3>"))
    display(df.head())

,gender,age,country,city,exp_group,os,source
user_id,,,,,,,
200,1,34,Russia,Degtyarsk,3,Android,ads
201,0,37,Russia,Abakan,0,Android,ads
202,1,17,Russia,Smolensk,4,Android,ads
203,0,18,Russia,Moscow,1,iOS,ads
204,0,36,Russia,Anzhero-Sudzhensk,3,Android,ads


,text,topic
post_id,,
1,UK economy facing major risks\n\nThe UK manufa...,business
2,Aids and climate top Davos agenda\n\nClimate c...,business
3,Asian quake hits European shares\n\nShares in ...,business
4,India power shares jump on debut\n\nShares in ...,business
5,Lacroix label bought by US firm\n\nLuxury good...,business


,timestamp,user_id,post_id,action,target
0,2021-11-27 12:14:54,86261,3686,view,1
1,2021-11-27 12:16:11,86261,3686,like,0
2,2021-11-27 12:16:13,86261,3828,view,1
3,2021-11-27 12:17:25,86261,3828,like,0
4,2021-11-27 12:17:27,86261,3462,view,0


,timestamp,user_id,post_id,action,target,gender,age,country,city,exp_group,os,source,text,topic
0,2021-11-16 16:58:47,51517,4998,view,0,0,23,Russia,Melnikovo,4,iOS,ads,Five Deadly Venoms is not as bloody and violen...,movie
1,2021-11-16 17:00:32,51517,3896,view,0,0,23,Russia,Melnikovo,4,iOS,ads,#Covid19 - Measured Responses? Those criticisi...,covid
2,2021-11-16 17:05:07,51517,1565,view,0,0,23,Russia,Melnikovo,4,iOS,ads,Mutu to hold talks with Juventus\n\nDisgraced ...,sport
3,2021-11-16 17:07:41,51517,7207,view,0,0,23,Russia,Melnikovo,4,iOS,ads,I remember seeing this film when I was 13 year...,movie
4,2021-11-16 17:11:52,51517,2676,view,0,0,23,Russia,Melnikovo,4,iOS,ads,@maddow Colleges should of known this was comi...,covid


## Transformation

In [4]:
# like garantee that view have
# a "like" guarantees that "view" was (see EDA section)
# for details
# so we need to drop all views that has same user_id and post_id
# with any like
X = joined_data.copy()

X = X.loc[
    X["action"].replace({
        "view" : 1, "like" : 0
    }).argsort()
].drop_duplicates(
    ["user_id", "post_id"],
    keep = "first"
)

X.drop(["user_id", "post_id"], axis = 1, inplace=True)

y = (X["action"] == "like").astype("int")
X.drop("action", axis = 1, inplace = True)

X["month"] = X["timestamp"].dt.month
X["year"] = X["timestamp"].dt.year
X["hour"] = X["timestamp"].dt.hour
X.drop("timestamp", axis = 1, inplace = True)

,timestamp,user_id,post_id,action,target,gender,age,country,city,exp_group,os,source,text,topic
3190,2021-12-17 07:18:09,141636,4050,like,0,1,24,Russia,Nizhniy Novgorod,4,iOS,organic,Did you know that people infected with the cor...,covid
788,2021-12-16 07:35:12,86265,5580,like,0,0,17,Russia,Surgut,0,iOS,ads,I am dumbfounded that I actually sat and watch...,movie
4666,2021-12-01 08:34:18,329,5777,like,0,1,24,Ukraine,Novohrad-Volynskyi,4,iOS,ads,Final Score: 1.8 (out of 10)After seeing Jay a...,movie
3260,2021-12-29 17:50:08,141636,2821,like,0,1,24,Russia,Nizhniy Novgorod,4,iOS,organic,Youre probably being sarcastic but can we plea...,covid
9289,2021-10-05 22:48:59,134676,6022,like,0,0,19,Russia,Labinsk,2,Android,organic,To Die For (1989) was just another d.t.v. feat...,movie
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3467,2021-10-30 14:17:05,86268,6489,view,0,0,19,Russia,Novorossiysk,3,Android,ads,Cute film about three lively sisters from Swit...,movie
3468,2021-10-30 14:21:15,86268,4112,view,0,0,19,Russia,Novorossiysk,3,Android,ads,The alcohol industry is certainly a determinan...,covid
3469,2021-10-30 14:22:32,86268,6549,view,0,0,19,Russia,Novorossiysk,3,Android,ads,The film has so much potential which was not d...,movie
3453,2021-10-27 08:02:56,86268,5862,view,0,0,19,Russia,Novorossiysk,3,Android,ads,Two hard-luck but crafty ladies decide to act ...,movie


In [30]:
X["target"].value_counts()

0    8598
Name: target, dtype: int64

## EDA

Actions types.

In [4]:
pd.Series(
    feed_data["action"].value_counts(),
    name = "number"
).to_frame()

,number
view,8848
like,1152


Let's check if record about like exclude record about view. Results show that it doesn't, so I need to exclude "view" records for any observation that has the same "like" record.

In [5]:
view_like_combs= feed_data.groupby(["user_id", "post_id"])["action"].apply(
    lambda actions: pd.Series({
        "like" : (actions == "like").any(),
        "view" : (actions == "view").any()
    })
).unstack()
pd.crosstab(
    view_like_combs["like"],
    view_like_combs["view"]
)

view,True
like,
False,7325
True,1137


In [28]:
numeric_columns = ['target', 'gender', 'age', 'exp_group']

In [29]:
joined_data.select_dtypes("object")

,action,country,city,os,source,text,topic
1488,like,Ukraine,Zaporizhzhia,Android,organic,6 months into #covid19 many companies like our...,covid
4084,like,Russia,Kursk,iOS,organic,When you think about it...they are. \n#furries...,covid
4086,like,Russia,Kursk,iOS,organic,Swap offer for pirated Windows XP\n\nComputer ...,tech
4090,like,Russia,Kursk,iOS,organic,Damp telling of the American Revolution.When f...,movie
1539,like,Russia,Saint Petersburg,iOS,ads,Did #RobertTrump die from #COVID19 ?,covid
...,...,...,...,...,...,...,...
3569,view,Russia,Ufa,iOS,ads,"The entire movie, an artful adaptation of one ...",movie
3570,view,Russia,Ufa,iOS,ads,"This movie was chosen, quite frankly as a pig ...",movie
3571,view,Russia,Ufa,iOS,ads,"A warm, touching movie that has a fantasy-like...",movie
3555,view,Russia,Ufa,iOS,ads,I first saw this in the theater in 1969 when I...,movie
